In [1]:
import pandas as pd
import numpy as np

In [102]:
df_adp = pd.read_csv('data/seasonADPs.csv')
df_rank = pd.read_csv('data/seasonTotals.csv')

In [103]:
# adjust df_adp pre-join
split_name = df_adp['Player'].str.split(' ', n=1, expand=True)
df_adp['Player'] = split_name[0].str[0] + '.' + split_name[1]

df_adjusted = None
for pos in ['RB','WR','QB','TE']:
    for season in df_adp['Season'].unique().tolist():
        # filter for one position and season at a time
        df_pos = df_adp.loc[df_adp['Pos']==pos].loc[df_adp['Season']==season]
        
        # rank by ADP ascending
        df_pos = df_pos.sort_values(by='ADPRank', ascending=True)
        df_pos['ADPPosRank'] = [i for i in range(1,len(df_pos.index)+1)]
        
        if df_adjusted is None:
            df_adjusted = df_pos.copy(deep=True)
        else:
            df_adjusted = pd.concat([df_adjusted, df_pos], ignore_index=True)

df_adp = df_adjusted[['Season','Player','Pos','ADPPosRank']]

In [104]:
# adjust df_rank pre-join
df_rank = df_rank[['Season','Player','FPts']]

In [105]:
df = df_adp.merge(df_rank, on=['Season','Player'], how='inner')
df.head()

,Season,Player,Pos,ADPPosRank,FPts
0,2010,C.Johnson,RB,1,334.8
1,2010,C.Johnson,RB,1,285.9
2,2010,C.Johnson,WR,4,334.8
3,2010,C.Johnson,WR,4,285.9
4,2010,C.Johnson,WR,17,334.8


In [106]:
df_check = df.groupby(['Season','Player']).agg('count')
df_check = df_check.loc[df_check['Pos']>1].reset_index()
exclude_list = df_check['Player'].unique()

df = df.loc[~df['Player'].isin(exclude_list)]
df.head()

,Season,Player,Pos,ADPPosRank,FPts
6,2010,A.Peterson,RB,2,289.9
7,2010,R.Rice,RB,3,295.9
8,2010,M.Jones-Drew,RB,4,244.0
9,2010,F.Gore,RB,5,228.5
10,2010,M.Turner,RB,6,233.5


In [107]:
# calculate positional ADP and final Ranking
df_adjusted = None
for pos in ['RB','WR','QB','TE']:
    for season in df['Season'].unique().tolist():
        # filter for one position and season at a time
        df_pos = df.loc[df['Pos']==pos].loc[df['Season']==season]
        
        # rank by final FPts descending
        df_pos = df_pos.sort_values(by='FPts', ascending=False)
        df_pos['FPtsRank'] = [i for i in range(1,len(df_pos.index)+1)]
        
        if df_adjusted is None:
            df_adjusted = df_pos.copy(deep=True)
        else:
            df_adjusted = pd.concat([df_adjusted, df_pos], ignore_index=True)

df = df_adjusted[['Season','Pos','ADPPosRank','FPtsRank','FPts']]
df.loc[df['Season']==2012].head()

,Season,Pos,ADPPosRank,FPtsRank,FPts
105,2012,RB,13,1,332.6
106,2012,RB,1,2,318.8
107,2012,RB,2,3,305.1
108,2012,RB,15,4,273.7
109,2012,RB,25,5,269.3


In [108]:
df.to_csv('data/ADPvsRank_Clean.csv', index=False)